In [ ]:
import tensorflow
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation
from keras.layers import Dropout, Flatten, Dense
from keras import backend as K

import os
import time
import numpy as np
import matplotlib
from tensorflow import keras


# In[14]:


input_size = 360,720

# import cv2

# cv2.imread('C:/Users/HP/Desktop/Pneumonia project dataset and code/Final data of project/Dataset after augmentation/Training_dataset/Healthy_data_Result/102_1b1_Ar_sc_Meditron.png')

train_data_dir = 'C:/Users/HP/Desktop/Italian_Dataset_Melspectrogram/E/Training'
test_data_dir = 'C:/Users/HP/Desktop/Italian_Dataset_Melspectrogram/E/Testing'

epochs = 15
batch_size = 32


# In[15]:


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1. /255,
    width_shift_range=0.2, # Shift the image horizontally by 20% of the width
    height_shift_range=0.2, # Shift the image vertically by 20% of the height
    zoom_range=0.2, # Zoom in the image by 20%
    fill_mode='nearest' # Fill any empty pixels with the nearest available pixel
)

test_datagen = ImageDataGenerator(rescale=1. /255 )
   
    
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(360, 720),                
    batch_size=batch_size,
    class_mode='binary')    


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(360, 720),
    batch_size=batch_size,
    class_mode='binary')


# In[16]:


y_true = test_generator.classes


# In[17]:


keras.backend.clear_session()
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape = (360, 720, 3)))  # 64 fliters with kernal_size= (3,3)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# In[18]:


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()


# In[19]:


keras.optimizers.Adam(
    learning_rate=0.0001,
)

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs
)

score = model.evaluate(train_generator)


# In[20]:


score = model.evaluate(test_generator)


# In[21]:


model.save("my_cnn4_model.h5")


# In[22]:


y_pred=(model.predict(test_generator)>0.5).astype(int)

# y_pred= (model.predict_generator(test.classes)>0.5).astype(int)


import matplotlib.pyplot as plt

import numpy

from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_true, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels

= [False, True])

cm_display.plot()

plt.show()





# In[23]:


from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print('Accuracy: %.3f' % accuracy_score(y_true, y_pred))
print('precision_score: %.3f' % precision_score(y_true, y_pred))
print(' recall_score: %.3f' %  recall_score(y_true, y_pred))
print('f1_score: %.3f' % f1_score(y_true, y_pred))


# In[24]:


import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['Accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(['loss'])
plt.plot(['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# In[25]:


from sklearn.metrics import matthews_corrcoef
print('MCC_score: %.3f' % matthews_corrcoef(y_true, y_pred))
#print('MCC_score: %, 3f' % matthews_corrcoef(y_true, y_pred))
